# Map mRNA CPM Stats by Regions and Taxa

In [ ]:
# Important to keep this in sync whenever renaming this notebook
notebook_name = 'mRNA_Maps'

# Import modules
import pymap as pm # SJS module: pymap.py must reside in current directory
import json
import shutil
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import matplotlib.patches as patches
import seaborn as sns
import zipfile
from matplotlib.patches import Rectangle, Polygon
from matplotlib.backends.backend_pdf import PdfPages

# Make some choices...
#dataset = 'Y21ss'
#dataset = 'Y21tenX'
dataset = 'Y23'
#dataset = 'H19'

if dataset[0] == 'Y': # Path for tables specific to species but not to dataset
    species = 'mouse'
    tables_path = 'Resources/Mouse_tables/'
else:
    species = 'human'
    tables_path = 'Resources/Human_tables/'    

# Top gene sets
#geneset = '20NPPs_byGa'
geneset = '20NPPs_byGa*Rs'
#geneset = '36NPRs_conjTop20NPPs'
#geneset = '37NPRs_conjTop20NPPs'
#geneset = '61NPPs'
#geneset = '30NPPs'
#geneset = '30NPPs_byGa'
#geneset = '20NPPs'
#geneset = 'Top20NPPs'
#geneset = '190GPCRs'
#geneset = 'Q11_GPCRs'
#geneset = 'S_GPCRs'
#geneset = 'IO_GPCRs'
#geneset = '77NPRs'
#geneset = 'Monoamine_Rs'
#geneset = '30NPPs*Rs'
#geneset = '20NPPs*Rs'
#geneset = '51_NP-GPCRs_Conj-Top30_NPPs'
#geneset = '155_Channels'
#geneset = 'Slitrks'

#type_id = 'All' # 'All', '01 Neuronal', '02 Non-Neuronal' (Y21), '02 Glial' (Y23)
type_id = '01 Neuronal' 
#type_id = '01 Neuronal' # 'All', '01 Neuronal', '02 Non-Neuronal' (Y21), '02 Glial' (Y23)
#type_id = '02 Glial' # 'All', '01 Neuronal', '02 Non-Neuronal' (Y21), '02 Glial' (Y23)

# Assemble the general filepaths
name_root = notebook_name
source_root = 'NP'
#source_root = 'nMod'
results_path = 'Products/'+name_root+'/'
pdf_path = results_path+'PDFs/'
#csv_path = results_path+'CSVs/'
zip_path = results_path+'ZIPs/'
temp_path = 'Temp/'

# Read annotated expression data
source_path = 'Resources/'+dataset+'_'+source_root+'_counts.csv.zip' # 'NP' or 'nMod'
support_path = 'Resources/'+dataset+'_'+source_root+'_support.json' # 'NP' or 'nMod'

print('Reading',source_path)
df = pd.read_csv(source_path)

# Filter based on type ID
if type_id != 'All': anno_counts = df[df['type'] == type_id]#.copy()
else: anno_counts = df#.copy() # IS "COPY" REALLY NECESSARY?

# Read support dictionary
with open(support_path, 'r') as fp:
    support_dict = json.load(fp)

# Extract annotation value dictionaries
region_colors = dict(zip(anno_counts['region'],anno_counts['region_color']))
class_colors = dict(zip(anno_counts['class'],anno_counts['class_color']))
class_colors = {key: class_colors[key] for key in sorted(class_colors)}
subclass_colors = dict(zip(anno_counts['subclass'],anno_counts['subclass_color']))
subclass_colors = {key: subclass_colors[key] for key in sorted(subclass_colors)}
subclass_classes = dict(zip(anno_counts['subclass'],anno_counts['class']))
subclass_classes = {key: subclass_classes[key] for key in sorted(subclass_classes)}
subclass_shorts = dict(zip(anno_counts['subclass'],anno_counts['subclass']))
subclass_shorts = {key: subclass_shorts[key].split(' ',1)[1] for key in sorted(subclass_shorts)}
cluster_colors = dict(zip(anno_counts['cluster'],anno_counts['cluster_color']))
cluster_colors = {key: cluster_colors[key] for key in sorted(cluster_colors)}

# Read CSV table to form region dictionaries
df = pd.read_csv('Resources/'+dataset+'_region_table.csv')
region_names = df.set_index('region_label')['region_name'].to_dict()
region_shorts = df.set_index('region_label')['region_short'].to_dict()
region_zones = df.set_index('region_label')['zone'].to_dict()
zone_regions = {}
for key, value in region_zones.items():
    if value not in zone_regions:
        zone_regions[value] = [key]
    else:
        zone_regions[value].append(key)

# Form annotation unique value lists
types = (list(set(anno_counts['type']))); types.sort()
regions = list(region_colors.keys())
classes = list(class_colors.keys())
subclasses = list(subclass_colors.keys())
clusters = list(cluster_colors.keys())

if dataset == 'Y23': # 'supertypes' in Y23 dataset only
    supertype_colors = dict(zip(anno_counts['supertype'],anno_counts['supertype_color']))
    supertype_colors = {key: supertype_colors[key] for key in sorted(supertype_colors)}
    supertypes = list(supertype_colors.keys())

if dataset[0:3] == "Y21": # 'neighborhood' in Y21 datasets only
    neighborhood_colors = dict(zip(anno_counts['neighborhood'],anno_counts['neighborhood_color']))
    neighborhood_colors = {key: neighborhood_colors[key] for key in sorted(neighborhood_colors)}
    neighborhoods = list(neighborhood_colors.keys())
    # Remove any undesired subclasses
    subclasses.remove('01 CR')
    subclasses.remove('02 Meis2')

# Read a table to form gene label dictionary
df = pd.read_csv(tables_path + 'All_nMods_labeled.csv', index_col='Symbol')
gene_labels = df.to_dict()['Label']

# Configure multiindex
major_annos = ['type','region','class','subclass','cluster','counts_sum']
all_annos = support_dict['annotation_labels']
minor_annos = [s for s in all_annos if s not in major_annos]
anno_counts.drop(columns=minor_annos, inplace=True)
anno_counts.set_index(major_annos, inplace=True)

# Read in a genelist and corresponding counts data
convert_to_aggrs = True if geneset[-3:] == '*Rs' else False
if convert_to_aggrs: geneset = geneset[0:-3]

#df = pd.read_csv('Resources/'+geneset+'.csv')
df = pd.read_csv(tables_path+geneset+'.csv')
genelist = list(df['Symbol'])  

#counts_sum = anno_counts.index.get_level_values('counts_sum') # ???
df = anno_counts#[genelist].copy()
df_sum = df.index.get_level_values('counts_sum')
cpm = 1e6 * (df.T / df_sum).T 

if convert_to_aggrs:
    # Convert an NPP matrix to an aggregated, conjugate NP-GPCR matrix
    assert 'NPP' in geneset, 'This cell requires an NPP gene list!'

    newlist_name = geneset+'*Rs'
    print('Converting', geneset, 'to', newlist_name)
    # Use conjugate pair table to generate aggregate NP_GPCRs conjugate to a set of NPPs
    # Read pair table
    pair_table = pd.read_csv(tables_path + 'NP_pairs.csv')
    pair_table['Pair Label'] = pair_table['NPP'] + '->' + pair_table['NP-GPCR']
    pair_id = list(pair_table['Pair ID'])
    pair_npps = list(pair_table['NPP'])
    pair_nprs = list(pair_table['NP-GPCR'])
    pair_labels = list(pair_table['Pair Label'])
    pair_table.set_index('Pair ID', inplace=True)
    # Build cognate NPP/aggregate-NP-GPCRs dictionary
    df = pair_table[['NPP','NP-GPCR']].set_index('NPP').copy()
    df['Aggs'] = df['NP-GPCR']+' '
    df = df.groupby('NPP').sum()
    aggs_dict = df['Aggs'].to_dict()
    for npp in aggs_dict: aggs_dict[npp] = aggs_dict[npp].split()
    # Build dataframe with aggregated NP-GPCRs conjugate to included NPPs
    npp_list = genelist
    df = cpm[npp_list].copy()
    for npp in npp_list: df[npp+'Rs'] = cpm[aggs_dict[npp]].sum(axis=1)
    npr_list = [x+'Rs' for x in npp_list]
    cpm = df.copy()

    geneset = newlist_name
    genelist = npr_list
    cpm = cpm[npr_list].copy()
else: cpm = cpm[genelist].copy()

print('Genelist:',geneset,', Counts matrix shape:', cpm.shape)
gene_order = genelist

In [ ]:
# Set up and make some choices

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

#row_norm = True
row_norm = False
log_floor = 1
yticklabels=genelist
slant = 90
draw_vlines = True
perc_gamma = 0.7

# columns metrics: 'peak','sum','mpc',mean','cell%','clus%'
# row metrics = ['sum'] # 'max', 'mean', 'sum'
metrics = [('cell%','max'),
           ('mean','max'),
           ('mpc','max'),
           #('peak','max'),
           #('sum','sum')
          ]

#allow_sort = True
allow_sort = False
allow_log = True
#allow_log = False
#row_stat='max' # 'max', 'mean', 'sum'
threshold = 1
peak_quantile = 0.99

# These matter only for clus% metric
cluster_thresh_factor = 0.5
min_cluster = 10

# Generate filename root
root = dataset
if type_id == 'All': root += '-all-'
elif type_id == '01 Neuronal': root += '-neur-'
else: root += type_id+'='
root += geneset+'-'

print('threshold =', threshold)


In [ ]:
# Map regions, write results to a PDF and a ZIP archive of CSVs

font_scale = 1.4
sns.set_theme(font_scale=font_scale, rc={'patch.linewidth': 0})

min_region = 10

# Form titling/naming handle

handle = root+'regs-' + sn

pdf_file = pdf_path+handle+'.pdf'
zip_file = zip_path+handle+'.zip'
temp_csv = temp_path+'temp.csv'
pdf = PdfPages(pdf_file)

idx = pd.IndexSlice
zip_mode = 'w'

#for cell_class in ['00 All']: # for quick short result
#for cell_class in ['01 Glutamatergic']: # for quick short result
for cell_class in ['00 All']+classes:
    df = cpm.loc[idx[types,regions,classes,subclasses], :]
    if cell_class != '00 All': df = df.xs(cell_class, level='class')
    df = df.groupby('region').filter(lambda x: len(x) >= min_region) # filter out small groups
    if len(df) == 0: continue
    
    for m, metric in enumerate(metrics):
        item_stat, row_stat = metric
        sort = allow_sort if m == 0 else False
        log = False if '%' in item_stat else allow_log
        
        if item_stat == 'sum': # Sum values
            X = df.groupby('region').sum().T

        if item_stat == 'mpc': # Mean of non-zero values
            hots = (df>=threshold).astype(int).groupby('region').sum().T
            X = df.groupby('region').sum().T / hots
            X.replace([np.inf, -np.inf], np.nan, inplace=True)

        if item_stat == 'mean': # Find mean values
            X = df.groupby('region').mean().T

        if item_stat == 'peak': # Find value at some high percentile
            X = df.groupby('region').quantile(peak_quantile).T

        if item_stat == 'cell%': # Find percent expressing by cell
            X = 100 * (df>=threshold).astype(int).groupby('region').mean().T

        if item_stat == 'clus%': # Find percent expressing by cluster NOTA BENE: use of cluster_thresh_factor
            pre = len(df)
            df = df.groupby('cluster').filter(lambda x: len(x) >= min_cluster) # filter out small clusters
            post = len(df)
            print(pre - post, 'cells filtered out of', pre, 'due due to cluster size <', min_cluster)
            cluster_mean = df.groupby(['region','cluster']).mean()
            cluster_hot = (cluster_mean >= cluster_thresh_factor * threshold).astype(int)
            cluster_frac = cluster_hot.groupby('region').mean()
            X = 100 * cluster_frac.T

        # Make two useful lists
        column_labels = [region_shorts[s] for s in list(X.columns)]
        #column_labels = [region_names[s] for s in list(X.columns)]
        column_colors = [region_colors[s] for s in list(X.columns)]

        # Get unit tallies
        cell_tally = df.groupby('region').count().iloc[:,0].T
        cell_tally.index = column_labels

        # Find zone boundaries if desired
        if draw_vlines:
            vlines = []; zone = []
            for i, region in enumerate(X.columns):
                if region_zones[region] != zone: vlines += [i]; zone = region_zones[region]
            vlines += [len(X.columns)]        
        else: vlines = None

        # Draw subclass heatmap
        title = handle+'\n'+' Class: '+cell_class+', Metric: '+item_stat
        title += '\nminR=' + str(min_region)
        if 'clus' in item_stat: title += 'minC='+str(min_cluster)
            
        if '%' in item_stat:
            gamma = perc_gamma
            title += ', Thr='+str(threshold)+', Gamma='+str(gamma)
            cbar_label = '%Positive'; units =' %'
        else:
            cbar_label = item_stat.capitalize()+'_CPM'; units = ''
            gamma = 1.0

        X.index = yticklabels
        X.columns = column_labels
        
        if m > 0: X = X.reindex(index=list(row_stats.index))
        
        row_stats = pm.draw_heatmap(X, cell_tally, column_colors, title, pdf, row_norm=row_norm, log=log,
                                    row_stat=row_stat, log_floor=log_floor, vlines = vlines, gamma=gamma,
                                    row_sort=sort, slant=slant, cbar=True, cbar_label=cbar_label, units=units)
        
        # Write row statistics to CSV file
        #row_stats.index.name = cell_class.split()[1]
        #row_stats.to_csv(csv_path+handle+'-'+row_stat+'-Cla'+cell_class.split()[0]+'.csv')

        # Append cell tally row and add to Zip file 
        X.loc['cells'] = cell_tally
        X.to_csv(temp_csv)
        with zipfile.ZipFile(zip_file, zip_mode) as zipf:
            arc_name = handle+'_'+item_stat+'_'+cell_class.split()[0]+'.csv'
            zipf.write(temp_csv, arcname=arc_name)
        zip_mode = 'a'    
        print(arc_name,'added to:',zip_file, '\n\n')

#
pdf.close(); print('Graphic to PDF:', pdf_file)


In [ ]:
# Map subclasses by region, write results to a PDF and a ZIP archive of CSVs

font_scale = 1.4
sns.set_theme(font_scale=font_scale, rc={'patch.linewidth': 0})

reg_subset = regions

# Set minimum subclass size according to dataset choice
min_subclass = 10 if 'ss' in dataset else 100

# Form titling/naming handle
handle = root+'subs-'+str(len(reg_subset))+'regs-' + sn 

pdf_file = pdf_path+handle+'.pdf'
zip_file = zip_path+handle+'.zip'
temp_csv = temp_path+'temp.csv'

pdf = PdfPages(pdf_file)

idx = pd.IndexSlice
zip_mode = 'w'

for region in reg_subset:
    if region[0] == '0': # Leading zero indicates one specific region
        df = cpm.loc[idx[types,region,classes,subclasses], :]
        region_label = region_shorts[region]
    else: # Otherwise, interpret as a list of regions to aggregate
        df = cpm.loc[idx[types,zone_regions[region],classes,subclasses], :]
        region_label = region
    pre = len(df)
    df = df.groupby('subclass').filter(lambda x: len(x) >= min_subclass) # filter out small subclasses
    post = len(df)
    print(pre - post, 'cells filtered out of', pre, 'due to subclass size <', min_subclass)
    if len(df) == 0: continue
        
    for m, metric in enumerate(metrics):
        item_stat, row_stat = metric
        sort = allow_sort if m == 0 else False
        log = False if '%' in item_stat else allow_log
        
        if item_stat == 'cell%': # Find percent of cell hot
            X = 100 * (df>=threshold).astype(int).groupby('subclass').mean().T

        if item_stat == 'mean': # Find mean values
            X = df.groupby('subclass').mean().T

        if item_stat == 'mpc': # Mean of non-zero values
            hots = (df>=threshold).astype(int).groupby('subclass').sum().T
            X = df.groupby('subclass').sum().T / hots
            X.replace([np.inf, -np.inf], np.nan, inplace=True)

        if item_stat == 'peak': # Find value at some high percentile
            X = df.groupby('subclass').quantile(peak_quantile).T

        if item_stat == 'sum': # Sum values
            X = df.groupby('subclass').sum().T

        if item_stat == 'clus%': # Find percent expressing by cluster NOTA BENE: use of cluster_thresh_factor
            pre = len(df)
            df = df.groupby('cluster').filter(lambda x: len(x) >= min_cluster) # filter out small clusters
            post = len(df)
            print(pre - post, 'cells filtered out of', pre, 'due due to cluster size <', min_cluster)
            cluster_mean = df.groupby(['subclass','cluster']).mean()
            cluster_hot = (cluster_mean >= cluster_thresh_factor * threshold).astype(int)
            cluster_frac = cluster_hot.groupby('subclass').mean()
            X = 100 * cluster_frac.T

        # Make two useful lists
        column_labels = [subclass_shorts[s] for s in list(X.columns)]
        column_colors = [subclass_colors[s] for s in list(X.columns)]

        # Get unit tallies
        cell_tally = df.groupby('subclass').count().iloc[:,0].T
        cell_tally.index = column_labels

        # Find zone boundaries if desired
        if draw_vlines:
            vlines = [0]; zone = False
            for i, subclass in enumerate(X.columns):
                if dataset == 'Y23':
                    # Make fine zone distinctions
                    #if subclass_classes[subclass] != zone:
                     #   vlines += [i]; zone = subclass_classes[subclass]  
                    # Make only coarse zone distinctions
                    if ('GLUT' in subclass.upper()) != zone:
                        print(zone, subclass)
                        vlines += [i]; zone = ('GLUT' in subclass.upper())
                else:
                    if subclass_classes[subclass] != zone:
                        vlines += [i]; zone = subclass_classes[subclass] 
            vlines += [len(X.columns)]
        else: vlines = None

        # Draw subclass heatmap
        title = handle[:-1]+'\n'+sn+' Region: '+region_label+' ('+item_stat+')'
        # if '%' in item_stat: cbar_label = '%Positive'; units =' %'
        #else: cbar_label = item_stat.capitalize()+'_CPM'; units = ''
            
        if '%' in item_stat:
            gamma = perc_gamma
            title += ', Thr='+str(threshold)+', Gamma='+str(gamma)
            cbar_label = '%Positive'; units =' %'
        else:
            cbar_label = item_stat.capitalize()+'_CPM'; units = ''
            gamma = 1.0
            
        X.index = yticklabels
        X.columns = column_labels
        
        if m > 0: X = X.reindex(index=list(row_stats.index))
        
        row_stats = pm.draw_heatmap(X, cell_tally, column_colors, title, pdf, row_norm=row_norm, log=log,
                                    row_stat=row_stat, vlines = vlines, gamma=gamma,
                                    row_sort=sort, slant=slant, cbar=True, cbar_label=cbar_label, units=units)

        # Append cell tally row and add to Zip file 
        X.loc['cells'] = cell_tally
        X.to_csv(temp_csv)
        with zipfile.ZipFile(zip_file, zip_mode) as zipf:
            arc_name = handle+'-'+item_stat+'_'+region.split()[1]+'.csv'
            zipf.write(temp_csv, arcname=arc_name)
        zip_mode = 'a'    
        print(arc_name,'added to:',zip_file,'\n\n')

#
pdf.close(); print('Graphic to PDF:', pdf_file)


In [ ]:
# for Y21ss: trim to isocortical regions if desired
#regions = ['001 VISp','002 VIS','005 PTLp','006 AUD','007 SSp','009 SSs','010 GU','011 MOp','012 ALM',
#           '014 CLA','016 PL-ILA','017 ORB','018 AI','019 ACA','020 RSP','021 TEa-PERI-ECT'] #,
           #'023 ENTm','024 ENTl','025 PAR-POST-PRE','027 SUB-ProS','028 HIP']
